# Test saved model

In [17]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import os

In [18]:
import pathlib

data_dir = pathlib.Path("/home/theorist/Desktop/Programming/ML/capstone-1/")

In [19]:
data_dir = data_dir / "train" / "images"
data_dir

image_count = len(list(data_dir.glob('*/*.jpg')))
image_count

1799

In [20]:
batch_size = 32
img_height = 180
img_width = 180

In [21]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 1799 files belonging to 6 classes.
Using 1440 files for training.


In [22]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 1799 files belonging to 6 classes.
Using 359 files for validation.


In [23]:
class_names = train_ds.class_names
print(class_names)

['crazing', 'inclusion', 'patches', 'pitted_surface', 'rolled-in_scale', 'scratches']


In [24]:
model = tf.keras.models.load_model("steel_defect_clf.keras")

In [25]:
crazing_path = pathlib.Path("/home/theorist/Desktop/Programming/ML/capstone-1/crazing_19.jpg")

img = tf.keras.utils.load_img(
    crazing_path, target_size=(img_height, img_width)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
This image most likely belongs to crazing with a 84.28 percent confidence.


# TFLite

In [27]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpc7fjr94r/assets


INFO:tensorflow:Assets written to: /tmp/tmpc7fjr94r/assets


Saved artifact at '/tmp/tmpc7fjr94r'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 180, 180, 3), dtype=tf.float32, name='input_layer_3')
Output Type:
  TensorSpec(shape=(None, 6), dtype=tf.float32, name=None)
Captures:
  123728936969216: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123728937016608: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123728937024880: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123728937021184: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123728937011504: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123728937175520: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123728937185200: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123728937389920: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123728937188192: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123728937392736: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1736757436.615625   40119 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1736757436.615637   40119 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-01-13 14:07:16.615743: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpc7fjr94r
2025-01-13 14:07:16.616226: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-01-13 14:07:16.616236: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpc7fjr94r
2025-01-13 14:07:16.620957: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-01-13 14:07:16.659898: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpc7fjr94r
2025-01-13 14:07:16.668638: I tensorflow/cc/saved_model/loader.cc:466] SavedModel load for tags { serve }; Status: success: OK. Took 52897 microseconds.


In [29]:
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

In [30]:
!ls

 model.tflite		 'Steel Industry.ipynb'
 steel_defect_clf.keras   tflite_config.ipynb


In [32]:
TF_MODEL_FILE_PATH = 'model.tflite'
interpreter = tf.lite.Interpreter(model_path=TF_MODEL_FILE_PATH)

In [33]:
interpreter.get_signature_list()

{'serving_default': {'inputs': ['input_layer_3'], 'outputs': ['output_0']}}

In [34]:
classify_lite = interpreter.get_signature_runner('serving_default')
classify_lite

In [37]:
predictions_lite = classify_lite(input_layer_3=img_array)['output_0']
score_lite = tf.nn.softmax(predictions_lite)

In [38]:
print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score_lite)], 100 * np.max(score_lite))
)


This image most likely belongs to crazing with a 84.28 percent confidence.


In [39]:
print(np.max(np.abs(predictions - predictions_lite)))

1.5258789e-05


In [1]:
!stat --printf="%s" model.tflite

15961432

In [2]:
!stat --printf="%s" steel_defect_clf.keras

47933199